In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.metrics import silhouette_score

In [ ]:
# 1. Завантаження та попередня обробка даних
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.data'
column_names = ['checking_account', 'duration_month', 'credit_history', 'purpose', 'credit_amount',
                 'savings_account', 'employment', 'installment_rate', 'personal_status', 'other_debtors',
                 'present_residence', 'property_magnitude', 'age', 'other_installment_plans',
                 'housing', 'number_credits', 'job', 'people_liable', 'telephone', 'foreign_worker', 'class']
data = pd.read_csv(url, header=None, names=column_names, delimiter=' ')

In [ ]:
# Замінюємо пропущені значення
data.replace({'?': np.nan}, inplace=True)

In [ ]:
# Окремо обробляємо категоріальні та числові дані
categorical_features = data.select_dtypes(include=['object']).columns
numerical_features = data.select_dtypes(exclude=['object']).columns

In [ ]:
# Пайплайн для числових даних
numerical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

In [ ]:
# Пайплайн для категоріальних даних
categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

In [ ]:
# Загальний пайплайн
preprocessor = ColumnTransformer([
    ('num', numerical_pipeline, numerical_features),
    ('cat', categorical_pipeline, categorical_features)
])

In [ ]:
X = preprocessor.fit_transform(data)

In [ ]:
# 2. Побудова моделей кластеризації
def perform_kmeans(X, n_clusters):
    kmeans = KMeans(n_clusters=n_clusters, random_state=0)
    labels = kmeans.fit_predict(X)
    return labels, kmeans.inertia_

In [ ]:
def perform_agglomerative(X, n_clusters):
    agglom = AgglomerativeClustering(n_clusters=n_clusters)
    labels = agglom.fit_predict(X)
    return labels

In [ ]:
# Вибір кількості кластерів
k_range = range(2, 11)
sse = []

In [ ]:
for k in k_range:
    _, inertia = perform_kmeans(X, k)
    sse.append(inertia)

In [ ]:
# Побудова графіка для методу ліктя
plt.figure(figsize=(12, 6))

In [ ]:
plt.subplot(1, 2, 1)
plt.plot(k_range, sse, marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('SSE')
plt.title('Elbow Method for Optimal k')

In [ ]:
# 3. Зменшення розмірності з PCA
pca = PCA(n_components=2)
X_reduced = pca.fit_transform(X)

In [ ]:
# Графік поясненої дисперсії
explained_variance_ratio = pca.explained_variance_ratio_

In [ ]:
plt.subplot(1, 2, 2)
plt.plot(range(1, len(explained_variance_ratio) + 1), explained_variance_ratio.cumsum(), marker='o')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.title('PCA - Explained Variance')

In [ ]:
plt.tight_layout()
plt.show()

In [ ]:
# 4. Кластеризація на зменшеному датасеті
optimal_k = 4  # Виберіть оптимальне значення на основі графіка ліктя

In [ ]:
# Кластеризація з K-середніми на зменшеному датасеті
labels_kmeans = perform_kmeans(X_reduced, optimal_k)[0]

In [ ]:
# Кластеризація з ієрархічним методом на зменшеному датасеті
labels_agglomerative = perform_agglomerative(X_reduced, optimal_k)

In [ ]:
# Оцінка кластеризацій
silhouette_kmeans = silhouette_score(X_reduced, labels_kmeans)
silhouette_agglomerative = silhouette_score(X_reduced, labels_agglomerative)

In [ ]:
print(f"Silhouette Score for K-means (Reduced Data): {silhouette_kmeans}")
print(f"Silhouette Score for Agglomerative (Reduced Data): {silhouette_agglomerative}")

In [ ]:
# 5. Кластеризація на оригінальному датасеті
labels_kmeans_original = perform_kmeans(X, optimal_k)[0]
labels_agglomerative_original = perform_agglomerative(X, optimal_k)

In [ ]:
# Оцінка кластеризацій на оригінальному датасеті
silhouette_kmeans_original = silhouette_score(X, labels_kmeans_original)
silhouette_agglomerative_original = silhouette_score(X, labels_agglomerative_original)

In [ ]:
print(f"Silhouette Score for K-means (Original Data): {silhouette_kmeans_original}")
print(f"Silhouette Score for Agglomerative (Original Data): {silhouette_agglomerative_original}")